# Sleeper Standings

Use [Sleeper API Documentation](https://docs.sleeper.com/) to complete this.

Sample League Sleeper IDs:
 1. QUESTASY Football Trial: ```865826998382112768```

TODO:
 - [x] Consider using [leg](https://api.sleeper.app/v1/state/nfl) for the "week" variable so you don't have to manually update it weekly
 - [X] Join [Rosters endpoint "owner_id"](https://api.sleeper.app/v1/league/844512859378528256/rosters) with [Users endpoint "user_id"](https://api.sleeper.app/v1/league/844512859378528256/users) to get "display_name" associated with overall stats.
 - [x] For weekly stats (where week < "leg") use the [matchups endpoint](https://docs.sleeper.com/#getting-matchups-in-a-league) and match user table's roster ID with matchup's roster ID

In [ ]:
## import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

##
## SETTINGS:
## List of Leagues
## Output: list_leagues (variable; list)
##

leagues = [999761008103067648,\
           1003091416626683904,\
           1003692357414719488
          ]


#
# Get the current week
#
# variable: num_week
#

url_nfl_state= 'https://api.sleeper.app/v1/state/nfl'
response = requests.get(url_nfl_state)
# It's -1 because I will usually pull rankings at the beginning of the following week.
current_week = response.json()['week']-1



#
# Join Rosters endpoint "owner_id" with Users endpoint "user_id" to get "display_name" associated with overall stats.
# Input: leagues
# Output table_teams; list_owners
#
# What we need:

# "Display Name": "username", "user_id": 1234, "roster_id": 4, "league_id":512321, "wins":5, "losses":5, ties:0 "points for": 234]

list_rosters_json = []
list_roster_data = []
list_users_data = []
list_matchups_data = []
i = 0

for league in leagues:
    #set display league id (i) to 1 and increment per league
    i+=1
    url_roster = f"https://api.sleeper.app/v1/league/{league}/rosters"
    roster_response = requests.get(url_roster)
    json_roster = roster_response.json()
    # do += instead of append so it keeps it flat
    list_rosters_json+=json_roster
    
#    leagueid, leagueid_display ownerId, rosterId,
    for team in json_roster:
        temp_list = [league, i, team['owner_id'], team['roster_id'], \
                     team['settings']['wins'],team['settings']['losses'],\
                     team['settings']['ties'], \
                     float(team['settings']['fpts']+team['settings']['fpts_decimal']*.01),\
                     float(team['settings']['fpts_against']+team['settings']['fpts_against_decimal']*.01)\
                    ]
        list_roster_data.append(temp_list)

#     get users endpoint
    url_users = f"https://api.sleeper.app/v1/league/{league}/users"
    users_response = requests.get(url_users)
    json_users = users_response.json()
    for team in json_users:
        temp_list = [team['user_id'],team['display_name']]
        list_users_data.append(temp_list)
        
# get matchup endpoint
    for week in range(1,current_week+1):
        url_matchups =f"https://api.sleeper.app/v1/league/{league}/matchups/{week}"
        matchups_response = requests.get(url_matchups)
        json_matchups = matchups_response.json()
        for matchup in json_matchups:
            list_matchups_data.append([league, matchup['roster_id'], matchup['points']])
        

df_roster = pd.DataFrame(list_roster_data, columns =['League ID', 'League', 'Owner ID', 'Roster ID', 'Wins', "Losses", "Ties", "Pts For", "Pts Against"])
display(df_roster)

df_users = pd.DataFrame(list_users_data, columns = ['Owner ID', "Display Name"]) 
display(df_users)

df_matchups = pd.DataFrame(list_matchups_data, columns = ["League ID", "Roster ID", "Pts For"])
df_matchups_2 = df_matchups.copy().drop("Roster ID", axis=1).groupby('League ID').describe()
df_matchups_3 = df_matchups.copy().groupby(["League ID", "Roster ID"]).sum().reset_index()
df_matchups_4 = df_matchups_3.copy().drop("Roster ID", axis=1).groupby("League ID").describe()

# display(df_matchups_2)

display(df_matchups_4)

df_grouped = df_matchups_3.groupby(['League ID', 'Roster ID']).describe()
df_grouped_2 = df_grouped.join(df_matchups_4, on="League ID",rsuffix=" League")
df_grouped_2.columns = ["_".join(col_name).rstrip('_') for col_name in df_grouped_2.columns.to_flat_index()]
df_grouped_2 = df_grouped_2.reset_index().drop(["Pts For_count", "Pts For League_count","Pts For League_min","Pts For League_25%","Pts For League_50%","Pts For League_75%","Pts For League_max"],axis = 1)
df_grouped_2 = df_grouped_2.assign(z_score = (df_grouped_2['Pts For_mean'] - df_grouped_2['Pts For League_mean'])/df_grouped_2['Pts For League_std'])
display(df_grouped_2)

df_final = df_roster.join(df_users.set_index('Owner ID'), on='Owner ID', validate='1:1')
df_final = pd.merge(df_final, df_grouped_2, on=['League ID', 'Roster ID'])
df_final = df_final.reindex(columns=["Display Name","League", "Wins", "Losses", "Ties", "Pts For", "Pts Against", "z_score"]).sort_values(by=["Wins","z_score"],ascending=[False,False])
df_final = df_final.reset_index()
df_final.index+=1
df_final = df_final.drop("index",axis=1)
display(df_final)

,League ID,League,Owner ID,Roster ID,Wins,Losses,Ties,Pts For,Pts Against
0,999761008103067648,1,865826147894730752,1,0,4,0,186.10,226.88
1,999761008103067648,1,999901112129974272,2,1,3,0,205.72,215.70
2,999761008103067648,1,1003141180391059456,3,3,1,0,235.48,245.32
3,999761008103067648,1,999878541351157760,4,1,3,0,211.56,296.78
4,999761008103067648,1,1003409657097990144,5,4,0,0,268.16,248.20
5,999761008103067648,1,870027212978610176,6,2,2,0,205.56,175.00
6,999761008103067648,1,1003664384036634624,7,4,0,0,291.34,212.96
7,999761008103067648,1,1003683765553135616,8,2,2,0,192.54,192.74
8,999761008103067648,1,466108799711703040,9,0,4,0,186.82,244.94
9,999761008103067648,1,1000055348318015488,10,3,1,0,244.32,169.08


,Owner ID,Display Name
0,466108799711703040,dustinkang
1,865826147894730752,mikeyg7
2,870027212978610176,great8fan
3,999878541351157760,nxt830855
4,999901112129974272,Huffybadger
5,1000055348318015488,brookelesser
6,1003141180391059456,NickPB
7,1003409657097990144,dcdragon
8,1003664384036634624,jroffe52
9,1003683765553135616,sareet


Pts For                                               \
                      count     mean        std     min      25%     50%   
League ID                                                                  
999761008103067648     10.0  222.760  35.980763  186.10  195.795  208.64   
1003091416626683904    10.0  261.384  25.960505  229.08  248.130  258.38   
1003692357414719488    10.0  232.920  19.880286  205.26  219.915  228.54   

                                      
                         75%     max  
League ID                             
999761008103067648   242.110  291.34  
1003091416626683904  268.550  324.76  
1003692357414719488  245.045  271.92

,League ID,Roster ID,Pts For_mean,Pts For_std,Pts For_min,Pts For_25%,Pts For_50%,Pts For_75%,Pts For_max,Pts For League_mean,Pts For League_std,z_score
0,999761008103067648,1,186.10,NaN,186.10,186.10,186.10,186.10,186.10,222.760,35.980763,-1.018878
1,999761008103067648,2,205.72,NaN,205.72,205.72,205.72,205.72,205.72,222.760,35.980763,-0.473586
2,999761008103067648,3,235.48,NaN,235.48,235.48,235.48,235.48,235.48,222.760,35.980763,0.353522
3,999761008103067648,4,211.56,NaN,211.56,211.56,211.56,211.56,211.56,222.760,35.980763,-0.311277
4,999761008103067648,5,268.16,NaN,268.16,268.16,268.16,268.16,268.16,222.760,35.980763,1.261785
5,999761008103067648,6,205.56,NaN,205.56,205.56,205.56,205.56,205.56,222.760,35.980763,-0.478033
6,999761008103067648,7,291.34,NaN,291.34,291.34,291.34,291.34,291.34,222.760,35.980763,1.906019
7,999761008103067648,8,192.54,NaN,192.54,192.54,192.54,192.54,192.54,222.760,35.980763,-0.839893
8,999761008103067648,9,186.82,NaN,186.82,186.82,186.82,186.82,186.82,222.760,35.980763,-0.998867
9,999761008103067648,10,244.32,NaN,244.32,244.32,244.32,244.32,244.32,222.760,35.980763,0.599209


,Display Name,League,Wins,Losses,Ties,Pts For,Pts Against,z_score
1,sturt,2,4,0,0,324.76,219.88,2.441247
2,AlexEJerome,3,4,0,0,271.92,216.86,1.961742
3,jroffe52,1,4,0,0,291.34,212.96,1.906019
4,dcdragon,1,4,0,0,268.16,248.20,1.261785
5,srivatsnarasimhan,3,4,0,0,245.70,218.88,0.642848
6,SwagDawgy,3,3,1,0,250.66,216.76,0.892341
7,brookelesser,1,3,1,0,244.32,169.08,0.599209
8,michaelzhao,3,3,1,0,243.08,215.54,0.511059
9,NickPB,1,3,1,0,235.48,245.32,0.353522
10,jmiller3,2,3,1,0,261.86,218.68,0.018336
